In [1]:
import pandas as pd
from datetime import datetime
import requests
import sys
import json
from pandas.io.json import json_normalize
from bs4 import BeautifulSoup
import time
import random
import operator
import re
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
def clean_text(txt):
    txt = re.sub("[^a-záéíóúñüäë]", " ", txt.lower())
    txt = re.sub(' +',' ', txt)
    return txt.strip().split()
def get_gender2(names):
    names = clean_text(names)
    names = [x for x in names if gender_list.get(x,'a') != 'a']
    gender ={'m':0, 'f':0, 'a':0}
    for i, name in enumerate(names):
        g = gender_list.get(name,'a')
        gender[g] += 1
        gender[g] += 2 if len(names) > 1 and i == 0 and g != 'a' else 0 
    gender['a'] = 0 if (gender['f']+gender['m']) > 0 else 1
    return max(gender.items(), key=operator.itemgetter(1))[0]
def df_to_dict(df, key_column, val_column):
    """convierte dos pandas series en un diccionario"""
    xkey = df[key_column].tolist()
    xval = df[val_column].tolist()
    return dict(zip(xkey,xval))
gender_list = pd.read_csv('nombres.csv')
gender_list = df_to_dict(gender_list, key_column='nombre', val_column='genero')

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--incognito")
#/Users/leopoldo.alarcon/Documents/scriptsRRSS/chromedriver 2
#https://pbpython.com/pandas-html-table.html
browser = webdriver.Chrome(options=chrome_options, executable_path=ChromeDriverManager().install())
browser.get('https://www.amazon.com/-/es/Extracto-c%C3%A1psulas-verificado-laboratorio-fabricado/product-reviews/B084R9X8RL/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews')
elem = browser.find_element_by_xpath("//*")
soup = BeautifulSoup(elem.get_attribute('innerHTML'),'html.parser')
rev = soup.find_all("",'a-section review aok-relative')
len(rev)
#pages = range(86,88,1)
data = pd.DataFrame(columns=['FECHA', 'LUGAR', 'USUARIO', 'RATE', 'TITULO', 'REVIEW', 'GENERO','PROYECTO',
       'PRODUCTO'])
pages = range(1,2,1)
url1='https://www.amazon.com/-/es/Extracto-c%C3%A1psulas-verificado-laboratorio-fabricado/product-reviews/B084R9X8RL/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews'
url2='&pageNumber='
producto = url1[len('https://www.amazon.com/'):len('https://www.amazon.com/')+url1[len('https://www.amazon.com/'):].find('/')].replace('+',' ')
browser.maximize_window()
for p in pages:
    desplazar = random.randint(400, 1200)
    browser.execute_script('window.scrollTo(1, '+str(desplazar)+');')
    time.sleep(2)
    urlr = url1+str(p)+url2+str(p)
    print(urlr)
    browser.get(urlr)
    elem = browser.find_element_by_xpath("//*")
    soup = BeautifulSoup(elem.get_attribute('innerHTML'),'html.parser')
    reviews = soup.find_all("div",'a-section review aok-relative')
    print(len(reviews),len(data),desplazar)
    for rev in reviews:
        #print(com.get_text(),'\n')
        fecha = rev.find_all("span",'a-size-base a-color-secondary review-date')[0].get_text()
        fechas = fecha.split('el',1)[1].strip(' ')
        lugar = fecha.split('el',1)[0].split('en',2)[1].strip(' ')
        autor = rev.find_all("span",'a-profile-name')[0].get_text()
        gen = get_gender2(autor)
        rate = rev.find_all("span",'a-icon-alt')[0].get_text()
        rate = rate.split('de',1)[0]
        #title = rev.find_all("",'a-size-base a-link-normal review-title a-color-base review-title-content a-text-bold')[0].find_all("span",'')[0].get_text()
        title = rev.find_all("a",'review-title')[0].text.strip('\n')
        #comen = rev.find_all("span",'a-size-base review-text review-text-content')[0].find_all("span",'')[0].get_text()
        #comen = comen.strip('\n')
        comen = rev.find_all("span",'a-size-base review-text review-text-content')[0].text.strip('\n').strip(' ')
        #comentario = pd.DataFrame([[fechas,lugar,autor,rate,title,comen,gen]])
        #data = data.append(comentario,ignore_index=True)
        data = data.append({'FECHA': fechas,'LUGAR':lugar,'USUARIO':autor,'RATE':rate,'TITULO':title,
                    'REVIEW':comen,'GENERO':gen,'PROYECTO':'SINZU','PRODUCTO':'Extracto de hoja de morera blanca'},ignore_index=True)
    sleep = random.randint(9, 21)
    #print(sleep)
    time.sleep(sleep)



====== WebDriver manager ======
Current google-chrome version is 101.0.4951
Get LATEST chromedriver version for 101.0.4951 google-chrome
Driver [/Users/raul.castillo/.wdm/drivers/chromedriver/mac64/101.0.4951.41/chromedriver] found in cache


https://www.amazon.com/-/es/Extracto-c%C3%A1psulas-verificado-laboratorio-fabricado/product-reviews/B084R9X8RL/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews1&pageNumber=1
10 0 587


In [2]:
data

,FECHA,LUGAR,USUARIO,RATE,TITULO,REVIEW,GENERO,PROYECTO,PRODUCTO
0,16 de octubre de 2020,Estados Unidos,Jayson B.,2.0,Look elsewhere.,I take this particular herb daily to help with...,m,SINZU,Extracto de hoja de morera blanca
1,16 de diciembre de 2020,Estados Unidos,lindael,3.0,I did not record my glucose readings.,"I am going to try this again, the last time I ...",a,SINZU,Extracto de hoja de morera blanca
2,21 de julio de 2020,Estados Unidos,Donna,5.0,Herb for DM,My Mother asked me to get this Herb for her be...,f,SINZU,Extracto de hoja de morera blanca
3,16 de diciembre de 2020,Estados Unidos,Andre,5.0,It is pure mulberry,I enjoy the fact that it's mulberry only but I...,m,SINZU,Extracto de hoja de morera blanca
4,4 de junio de 2020,Estados Unidos,Betsy Perry,5.0,"So far, so good.",I take white mulberry leaf extract to help wit...,f,SINZU,Extracto de hoja de morera blanca
5,28 de julio de 2020,Estados Unidos,Amazon Customer,5.0,Mulberry is natural and good for lowering bloo...,"I like the fact that this is natural, affordab...",a,SINZU,Extracto de hoja de morera blanca
6,9 de julio de 2020,Estados Unidos,Amazon Customer,5.0,Important for blood sugar,Mulberry Leaf helps lower blood sugar . This ...,a,SINZU,Extracto de hoja de morera blanca
7,1 de junio de 2021,Estados Unidos,Annonymous,3.0,hmmm,I take this every day but not sure if it's doi...,a,SINZU,Extracto de hoja de morera blanca
8,16 de noviembre de 2021,Estados Unidos,VONCILLE S VANOVER,4.0,Great product,Great product,a,SINZU,Extracto de hoja de morera blanca
9,11 de diciembre de 2020,Estados Unidos,Paul Segal,5.0,good product,great for many things,m,SINZU,Extracto de hoja de morera blanca


In [5]:
export_csv = data.to_csv ("Gris6.csv", index = None, header=True , encoding='UTF-8')